En este programa iniciando en w=(10,-5,0), emisor: Source=(0,0,2000) y un receptor a la misma altura: Receiver=(2000,2000,2000) se ve cómo para un modelo de temperatura que decrece con la altura , la trayectoria se curva hacia abajo. En este caso concreto se obtiene un traveltime de 8.265 s y un tiempo computacional de 199.35 s. Ángulos polar θ=90.69754464285714º ( ángulo de emisión -0.69754464285714º )  y azimutal φ=46.81211545277572º con una precisión del 99.96405701333224 %.


El programa resuelve n=( 4 * 50(ángulos theta) * 20(ángulos phi) * l(número iteraciones, que a veces no alcanza el máximo) ) ecuaciones. Podría optimizarse resolviendo en parálelo o acotando más los ángulos theta y phi (teóricamente o a partir de datos experimentales con técnicas como ML)

In [2]:
using LinearAlgebra
using DifferentialEquations
using Plots
println("Este programa reconstruye la trayectoria de un rayo acústico entre un emisor y un receptor dados. Se sigue un modelo de viento constante y un modelo termodinámico para la velocidad del sonido. Se supone una caída de la temperatura de un grado centígrado por cada 200 m de altitud.")
function solicitar_vector(mensaje::String)
    println(mensaje)
    return parse.(Float64, split(readline(), ","))
end

function solicitar_flotante(mensaje::String)
    println(mensaje)
    return parse(Float64, readline())
end

v_wind = solicitar_vector("Introduce el vector velocidad del viento en metros por segundo como valores separados por comas (por ejemplo, 10.,-5.,0):")
Source = solicitar_vector("Introduce las coordenadas del emisor en metros como valores separados por comas (por ejemplo, 0.,0.,100.):")
Receiver = solicitar_vector("Introduce las coordenadas del receptor en metros como valores separados por comas (por ejemplo, 150.,150.,221.):")
T = solicitar_flotante("Introduce la temperatura a nivel de mar, relativa a la posición XY del emisor, en grados centígrados(por ejemplo, 15.):")

@time begin
function phi(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, v_wind::Array{Float64, 1}, tau) 
        tgphi= (Point2[2]-Point1[2]- v_wind[2]*tau)/(Point2[1]-Point1[1]- v_wind[1]*tau) #Cálculo phi primera iteración
        phi=rad2deg(atan(tgphi))
        return phi
end
function Slowness_Spherical(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, v_wind::Array{Float64, 1}, phi, c_source)
    x1,y1,z1=Point2-Point1 #Obtención parámetros primera iteración
    r=norm(Point2-Point1)
    theta=rad2deg(acos(z1/r))
    q0=1/(c_source + (v_wind[1]*cosd(phi)+ v_wind[2]*sind(phi))*cosd(theta) )
    qx=q0*cosd(phi)*sind(theta)
    qy=q0*sind(phi)*sind(theta)
    qz=q0*cosd(theta)
    return q0,qx,qy,qz,theta
end
function c_z(T,z) 
    if z>0
        c_air=331*sqrt((T+273.15-z/200)/273.15) #Modelo termodinámico v_sonido.Introduzco T_Z0 en grados centígrados por comodidad
    else
        c_air=331*sqrt((T+273.15)/273.15)
    end
    return c_air
end
function g!(dg, g, p, t)
    qx,qy = p
    q_aux = sqrt(qx^2 + qy^2 + g[4]^2)
    c=c_z(T,g[3])
    dg[1] = c*qx/q_aux + v_wind[1] #Variación coordenada X
    dg[2] = c*qy/q_aux + v_wind[2]  #Variación coordenada Y
    dg[3] = c*g[4]/q_aux + v_wind[3] #Variación coordenada Z
    dg[4]=+q_aux*(331^2/(400*273.15))/c #Variación qz
end
#### Necesitamos una velocidad inicial, correspondiente a la fuente, y un ángulo azimutal para empezar a iterar.
c_source=c_z(T, Source[3])
####
R=norm(Receiver-Source)
t_min=R/c_z(T,min(Source[3],Receiver[3]))
t_max=R/c_z(T, max(Source[3],Receiver[3]))
phi1=phi(Source,Receiver,v_wind, t_max) #Acotación teórica phi
phi2=phi(Source,Receiver,v_wind, t_min) #Acotación teórica phi
phi_i=(phi1+phi2)/2 #La media de la cota de phi como phi para la primera iteración
#######
q0,q_x,q_y,q_z,theta_i=Slowness_Spherical(Source,Receiver,v_wind, phi_i, c_source) #Obtención parámetros
#######
traveltime=[]
for l in 1:10
    thetas= range(theta_i-25/(2^(l-1)), theta_i+25/(2^(l-1)), 50); #Barrido theta. Se podría acotar más, teóricamente o con datos (ML u otras técnicas)
    if l==1
        phis=(min(phi1,phi2):0.05:max(phi1,phi2)) #Barrido phi. Se podría acotar aún más, con datos (ML u otras técnicas)
    else
        phi_distance=max(phi1,phi2)-min(phi1,phi2)
        phis= range(phi_i-phi_distance/(2^(l)), theta_i-phi_distance/(2^(l)), 20);
    end
    ###########
    angulos_theta = []
    angulos_phi = []
    tiempos = []
    for i in 1:length(thetas)
        for j in 1:length(phis)
            if l>1
                q0=1/(c_source + (v_wind[1]*cosd(phi_i)+ v_wind[2]*sind(phi_i))*cosd(theta_i) ) #Nuevos parámetros iniciales
            end
            q_x=q0*cosd(phis[j])*sind(thetas[i])
            q_y=q0*sind(phis[j])*sind(thetas[i])
            q_z=q0*cosd(thetas[i])
            # Definir los parámetros
            p = [q_x, q_y]
            u0 = [Source[1],Source[2],Source[3], q_z]
            tspan=[0,t_max] #Tiempo lo suficientemente grande
            # Crear el problema ODE
            prob = ODEProblem(g!, u0, tspan, p)
            # Resolver el problema ODE
            paso=1e-2*R/2000
            sol = solve(prob, RK4(), dt=paso, saveat=0:paso:t_max)
            # Comprobar si alguna solución está cerca del receptor. 
            #Rodeamos al receptor de una bola, cuyo tamaño se reduce a cada iteración.
            for k in 1:length(sol.u)
                if sqrt((sol.u[k][1] - Receiver[1])^2 +(sol.u[k][2] - Receiver[2])^2 + (sol.u[k][3] - Receiver[3])^2 ) < R/(20*2^(l-1))
                    push!(angulos_theta, thetas[i])
                    push!(angulos_phi, phis[j])
                    push!(tiempos, sol.t[k])
                    break
                end
            end
        end
    end
    if length(tiempos)==0
        if l==1
            println("No se encontró ninguna trayectoria entre fuente y receptor. Valora si estás en una zona de sombra o ajusta el paso de tiempo ")
        end
        break
    else
        traveltime = findmin(tiempos)
        indice = argmin(tiempos)
        theta_i=angulos_theta[indice]
        phi_i=angulos_phi[indice]
    end
end
q_x=q0*cosd(phi_i)*sind(theta_i)
q_y=q0*sind(phi_i)*sind(theta_i)
q_z=q0*cosd(theta_i)
# Definir los parámetros
p = [q_x, q_y]
u0 = [Source[1],Source[2], Source[3], q_z]
tspan=[0,traveltime[1]]
# Crear el problema ODE
prob = ODEProblem(g!, u0, tspan, p)
# Resolver el problema ODE
paso=1e-2*R/2000
sol = solve(prob, RK4(), dt=paso, saveat=0:paso:traveltime[1])
println("El traveltime es: ", traveltime[1], " s.") 
println("Se reconstruye la trayectoria con ángulos de emisión: polar θ=", theta_i, "º y azimutal φ=", phi_i, "º.")
error_relativo=(sqrt((sol.u[end][1] - Receiver[1])^2 +(sol.u[end][2] - Receiver[2])^2 + (sol.u[end][3] - Receiver[3])^2 )/R)*100
println("El error relativo es del ", error_relativo, " %.")
end
using Plots

x_data = [u[1] for u in sol.u]
y_data = [u[2] for u in sol.u]
z_data = [u[3] for u in sol.u]

# Crear el primer subplot (distancia puntos a recta)
# Normalizar el vector de dirección
dir_vector = (sol.u[end][1:3]-Source)/norm(sol.u[end][1:3]-Source)
# Calcular las distancias de los puntos a la línea recta
distances = [norm(cross(dir_vector, collect(point) - Source)) for point in zip(x_data, y_data, z_data)]
p1 = plot(z_data, distances, xlabel="Height (m)", ylabel="Distance to the straight line (m)",  label=false, color=:black, title="Distance of the points to the straight line")

# Crear el segundo subplot (plano XY)
p2 = plot(x_data, y_data, xlabel="x (m)", ylabel="y (m)", label=false, color=:red, title="XY plane")
scatter!([Source[1]], [Source[2]], markersize=1, color=:blue, label="Source")
scatter!([Receiver[1]], [Receiver[2]], markersize=1, color=:green, label="Receiver")

# Crear el tercer subplot (plano XZ)
p3 = plot(x_data, z_data, xlabel="x (m)", ylabel="z (m)", label=false, color=:red, title="XZ plane")
scatter!([Source[1]], [Source[3]], markersize=1, color=:blue, label="Source")
scatter!([Receiver[1]], [Receiver[3]], markersize=1, color=:green, label="Receiver")

# Crear el cuarto subplot (plano YZ)
p4 = plot(y_data, z_data, xlabel="y (m)", ylabel="z (m)", label=false, color=:red, title="YZ plane")
scatter!([Source[2]], [Source[3]], markersize=1, color=:blue, label="Source")
scatter!([Receiver[2]], [Receiver[3]] , markersize=1, color=:green, label="Receiver")
    

# Mostrar los cuatro subplots
plot(p1, p2, p3, p4, layout=(4, 1), size=(500, 1000))

#Guardar gráficas
#savefig(p1, "C:\\Users\\Miguel\\Desktop\\TFM\\Distancetothestraightline.png")
#savefig(p2, "C:\\Users\\Miguel\\Desktop\\TFM\\XYplane.png")
#savefig(p3, "C:\\Users\\Miguel\\Desktop\\TFM\\XZplane.png")
#savefig(p4, "C:\\Users\\Miguel\\Desktop\\TFM\\YZplane.png")

Este programa reconstruye la trayectoria de un rayo acústico entre un emisor y un receptor dados. Se sigue un modelo de viento constante y un modelo termodinámico para la velocidad del sonido. Se supone una caída de la temperatura de un grado centígrado por cada 200 m de altitud.
Introduce el vector velocidad del viento en metros por segundo como valores separados por comas (por ejemplo, 10.,-5.,0):
stdin> 10,-5,0
Introduce las coordenadas del emisor en metros como valores separados por comas (por ejemplo, 0.,0.,100.):
stdin> 0,0,2000
Introduce las coordenadas del receptor en metros como valores separados por comas (por ejemplo, 150.,150.,221.):
stdin> 2000,2000,2000
Introduce la temperatura a nivel de mar, relativa a la posición XY del emisor, en grados centígrados(por ejemplo, 15.):
stdin> 23
El traveltime es: 8.259007204258875 s.
Se reconstruye la trayectoria con ángulos de emisión: polar θ=90.65369897959184º y azimutal φ=46.81211545277572º.
El error relativo es del 0.12492966127436

"C:\\Users\\Miguel\\Desktop\\TFM\\YZplane.png"

In [3]:
sol.u[end]

4-element Vector{Float64}:
 1997.568966324387
 1998.8146780750712
 2002.273988437794
    3.847859100711921e-5

### Distancia fija: (0,0,2000) a (1000,1000,2000) , temperatura (23º) y viento fijos (10,-5,0)

Variando paso de tiempo: 

<div style="text-align: center;">
    
| Paso de tiempo(s) | Paso de tiempo para guardar los resultados (s) | Traveltime (s)   |Tiempo computacional (s) | Error Relativo |
|---------------------------------|----------------------------------|------------------|-------------------------|-----------|
|       1e-01                     |   1e-01                          |4.1000000000000005|     0.461               |0.011515   |
|       1e-02                     |   1e-01                          |4.1000000000000005|     0.475               |0.011515   |
|       1e-02                     |   1e-02                          | 4.13             |     5.501               |0.001183   |
|       1e-03                     |   1e-02                          | 4.13             |     5.598               |0.001183   |
|       1e-03                     |   1e-03                          | 4.133            |     69.296              |0.000377   |
|       1e-04                     |    1e-03                         | 4.133            |     69.006              |0.000377   |
|       1e-04                     |    1e-04                         |4.1324000000000005|     572.518             |0.000382   |

</div>

Vemos que los resultados solo dependen del paso de tiempo para guardar los resultados.
Un paso de tiempo de 1e-2 para guardar los resultados parece aceptable ya que se obtiene un traveltime con precisión hasta la centésima de segundo.



# Variamos la distancia 
Fuente, temperatura y viento fijos, los anteriores.

<div style="text-align: center;">
    
| Receptor (m)          | Distancia (m) | Traveltime (s)   |Tiempo computacional (s) | Error Relativo |
|-----------------------|---------------|------------------|-------------------------|----------------|
|       (100,0,2000)    |   100         |2.87              |     3.586               |0.001525        |
|       (1000,1000,3000)|   1732.05     |5.09              |     5.761               |0.001497        |
|       (3000,3000,3000)|   4358.90     |12.8              |     21.317              |0.000175        |
|       (5000,5000,0)   |   7348.47     |21.3              |     31.587              |0.000293        |
|    (10000,10000,10000)|   16248.08    |49.17             |     46.911              |0.003071        |
|    (30000,30000,0)    |   42473.52    |122.43            |     201.068             |0.000089        |
    
</div>



Nota: La trayectoria pasa en dos casos por debajo del 0, lo que se arregla al añadir reflexiones con el terreno.


# Paso de tiempo proporcional a la distancia.

Paso de tiempo de 1e-02 para 2km de distancia, a partir de ahí establecemos una proporción.

<div style="text-align: center;">
    
| Receptor (m)          | Distancia (m) | Paso (s)  | Traveltime (s)      |Tiempo computacional (s) | Error Relativo |
|-----------------------|---------------|-----------|---------------------|-------------------------|-----------|
|       (100,0,2000)    |   100         | 5e-04     |0.28650000000000003  |     10.756              |0.001547   |
|       (1000,0,2000)   |   1000        | 5e-03     |       2.865         |     10.721              |0.001488   |
|       (10000,0,2000)  |   10000       | 5e-02     |28.650000000000002   |     11.916              |0.000739   |
|       (100000,0,2000) |   100000      | 5e-01     |290.0                |     11.461              |0.001286   |  
|       (0,0,12000)     |   10000       | 5e-02     |30.950000000000003   |     16.199              |0.000092   | 
|       (0,0,22000)     |   20000       | 1e-01     |65.4                 |     17.393              |0.000089   |

</div>
